## install packages

In [1]:
!pip install tensorlayer
!pip install rawpy
!pip install easydict
!pip install numba

## Get utils from github

In [2]:
! git clone https://github.com/wanghoppe/combined_py.git

fatal: destination path 'combined_py' already exists and is not an empty directory.


## Mount Google drive
- if restart session, run from here

In [3]:
from google.colab import drive
# from google.colab import files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
work_dir = '/content/gdrive/My Drive/combine' # set working dir

In [0]:
from __future__ import division
import os, scipy.io, time, pickle, random
import tensorflow as tf
# import tensorflow.contrib.slim as slim
import numpy as np
import rawpy
# import glob
import tensorlayer as tl
from tensorlayer.prepro import imresize


from combined_py.dark_utils import *

# import os, time, pickle, random, time
from datetime import datetime
from time import localtime, strftime
import logging, scipy
from combined_py.model2 import SRGAN_g, SRGAN_d, Vgg19_simple_api
from combined_py.utils2 import *
from combined_py.config2 import config, log_config

from PIL import Image
from numba import cuda


In [0]:
RATIO = 20
INPUT_IMG = work_dir + '/test_data/sony_a7s_ii_38.arw'

In [7]:
INPUT_IMG

'/content/gdrive/My Drive/combine/test_data/sony_a7s_ii_38.arw'

## Load the first checkpoint

In [8]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess1 = tf.Session(config=config)

in_image = tf.placeholder(tf.float32, [None, None, None, 4])
out_image = network(in_image)
saver = tf.train.Saver()
sess1.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(work_dir + '/checkpoint1/Sony')
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess1, ckpt.model_checkpoint_path)

loaded /content/gdrive/My Drive/combine/checkpoint1/Sony/model.ckpt
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/combine/checkpoint1/Sony/model.ckpt


## Run the first model

In [0]:
raw = rawpy.imread(INPUT_IMG)
input_full = np.expand_dims(pack_raw(raw), axis=0) * RATIO
input_full = np.minimum(input_full, 1.0)
output = sess1.run(out_image, feed_dict={in_image: input_full})
output = np.minimum(np.maximum(output, 0), 1)

output = output[0, :, :, :]
output = output * 255

In [0]:
# tf.reset_default_graph()
# sess1.close()

In [0]:
# sess1.close()
# cuda.select_device(0)
# cuda.close()

## Load the second checkpoint

In [0]:
t_image = tf.placeholder('float32', [1, None, None, 3], name='input_image')
net_g = SRGAN_g(t_image, is_train=False, reuse=False)

sess2 = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess2)


[TL] InputLayer  SRGAN_g/in: (1, ?, ?, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (1, ?, ?, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b

In [0]:
tl.files.load_and_assign_npz(sess=sess2, name= work＿dir + '/checkpoint2' + '/g_srgan.npz', network=net_g)

## Check GPU usage

In [0]:
!nvidia-smi

## Run the second model

(first resize the image to 1/4 of the original)

In [0]:
output1 = imresize(output, (int(output.shape[0]/8), int(output.shape[1]/8)))
output1 = (output1 / 127.5) - 1
output2 = sess2.run(net_g.outputs, {t_image: [output]})

## save the output

In [0]:
tl.vis.save_image(output2[0], work_dir + '/output' + '/valid_gen444.png')